## Hybrid Filtering

In [86]:
from recommendation_data import dataset
from math import sqrt
import pandas as pd
import numpy as np

In [154]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [155]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [156]:
df=pd.read_csv('data_contents.csv',delimiter=';')
df1=pd.DataFrame(dataset)

In [157]:
df

,Film,Description
0,Ada Apa dengan Cinta 2,Budi Tom Romance ID
1,Aladdin,Budi Cruise Romance US
2,Avengers: End Game,Andi Michael Action US
3,Bumi Manusia,Charles Tom Drama ID
4,Captain Marvel,Andi Cruise Action US
5,Dilan 1991,Budi Tom Romance ID
6,Dua Garis Biru,Charles Tom Drama ID
7,Gundala,Andi Cruiser Action ID
8,Spiderman: Far From Home,Charles Michael Action US
9,The Lion King,Budi Michael Drama US


In [158]:
df1

,ANI,AhokTemanFirli,Damar Teman Firli,Dpv,Febi ganteng gak ada obat,Genjeh,Hania,Indra 1991 SM,Indra Junior,Jawaharal,...,Putrisqiana,Rima,Romantika,Star,Topik Zulkarnain,bunga,faizah,franadek,jul,luck
Ada Apa dengan Cinta 2,4,0,5,5,4,5,3,0,4,2,...,4,5,5,4,0,0,3,4,0,3
Aladdin,4,0,0,0,5,5,0,0,5,5,...,0,5,0,5,0,5,0,5,3,0
Avengers: End Game,0,3,5,5,5,5,0,0,5,5,...,5,5,0,5,5,5,5,5,3,4
Bumi Manusia,5,0,0,0,0,0,4,0,0,0,...,4,4,0,0,0,0,5,5,0,0
Captain Marvel,4,4,0,5,4,4,0,0,5,4,...,3,5,0,5,2,5,0,4,3,2
Dilan 1991,4,0,0,4,4,3,4,0,0,3,...,2,5,5,0,0,4,5,4,0,0
Dua Garis Biru,0,0,0,0,0,0,4,0,4,5,...,3,3,0,0,0,0,4,3,3,0
Gundala,0,0,0,4,3,4,5,5,0,4,...,3,5,0,4,0,4,0,4,3,0
Spiderman: Far From Home,3,0,5,5,5,4,0,0,5,5,...,4,5,0,0,4,5,0,4,3,0
The Lion King,0,0,0,0,0,0,0,0,5,4,...,0,4,0,5,5,5,0,4,3,0


In [170]:
def content_based(person,min_content_score):
    
    k=0
    not_watch=[]
    for i in df1[person]:
        if i==0:
            not_watch.append(df1.index[k])
            k=k+1
        else:
            k=k+1
    
    tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')
    tfidf_matrix=tf.fit_transform(df['Description'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    new=pd.DataFrame(cosine_similarities,index=df['Film'],columns=df['Film'])
    
    final=pd.DataFrame(new[not_watch].mean().sort_values(ascending=False),columns=['Score'])
    final2=final[final.Score>=min_content_score]
    indeks=final2.index
    
    return indeks

In [171]:
def person_correlation(person1, person2):

   # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Checking for ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

In [172]:
def user_recommendations(person,min_content_score):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    

    new_rankings=[]
    for i in rankings:
        if i[1] in content_based(person,min_content_score):
            new_rankings.append(i)
        
    return new_rankings

In [173]:
def hybrid_filtering(person,min_content_score):
    content_based(person,min_content_score)
    return user_recommendations(person,min_content_score)
    

In [179]:
hybrid_filtering('Dpv',0.18)

[(1.054009025444668, 'Dua Garis Biru'), (0.81908510791458, 'Bumi Manusia')]